**Using traffic images V1 API from data.gov.sg instead of LTA DataMall traffic images V2 API, because with V1 we can set the datetime parameter**

In [2]:
import requests
import os
import cv2
import json
import numpy as np
import pandas as pd
import collections
from datetime import datetime, timedelta

In [4]:
headers = { 'AccountKey' : 'ZSRd6ixqSy+V+GnHTV7/iQ==',
             'accept' : 'application/json'} 

**For the traffic images, we can actually query it for a specific timeframe by including the datetime as a parameter, meaning it doesn't have to be real-time. <br> <br> Ideally I would want to get the data for 365 days. The problem is that would simply take too long. <br> 1 query (90 pictures) takes around 40 seconds in average (based on %timeit). If we use 5 minutes interval, that would mean 40 x 12 = 480 seconds, 8 minutes for 1 hour of data. Times that by 24 hours, and it would take 192 hours, around 3 hours for only 1 day of data. <br> <br> Of course that is alright, but it would be nicer if we could make the processing faster, and get data for 1 month. <br> The long processing time is because we need to perform object detection, we can use a faster model but that would sacrifice the accuracy of the detection**

In [5]:
def get_images(date):
    
    image_url = "https://api.data.gov.sg/v1/transport/traffic-images"
    
    params = {"date_time": date.strftime("%Y-%m-%dT%H:%M:%S+08:00")}
    
    response = requests.get(image_url, params=params)
    
    #filename = datetime_value.strftime('%Y-%m-%d_%H-%M-%S.jpg')
    
    if response.json()["items"]:
        for i in response.json()["items"]: 
            for camera in i["cameras"]:                       
                image = requests.get(camera["image"])
                camera_id = camera["camera_id"]
                timestamp = camera["timestamp"]
                
                datetime_object = datetime.strptime(camera["timestamp"], "%Y-%m-%dT%H:%M:%S+08:00")
                datetime_new_format = datetime_object.strftime("%Y-%m-%d %H-%M-%S")
                filepath = "./dataset/"+datetime_new_format+" "+camera["camera_id"]+".jpg"
                
                pic_url = "image.jpg"
                
                file = open(pic_url, "wb")
                file.write(image.content)
                file.close()
                
                from_static_image(camera_id, timestamp)
                
                os.remove(pic_url)
                
                '''
                If we simply want to save the image files:
                
                datetime_object = datetime.strptime(camera["timestamp"], "%Y-%m-%dT%H:%M:%S+08:00")
                datetime_new_format = datetime_object.strftime("%Y-%m-%d %H-%M-%S")
                filepath = "./dataset/"+datetime_new_format+" "+camera["camera_id"]+".jpg"
                
                file = open(filepath, "wb")
                file.write(image.content)
                file.close()
                '''

In [6]:
# Detection confidence threshold
confThreshold = 0.2
nmsThreshold = 0.2

# Middle cross line position
middle_line_position = 225   
up_line_position = middle_line_position - 15
down_line_position = middle_line_position + 15

# Store Coco Names in a list
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')
print(len(classNames), classNames)

# Model Files
modelConfiguration = 'yolov3-320.cfg'
modelWeights = 'yolov3.weights'

# Load the network model
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
# Configure the network backend
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define random color for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

def postProcess(outputs, img):
    required_class_index = [2, 3, 5, 7]
    detected_classNames = []
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []

    for output in outputs:
        for det in output:
            scores = det[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if classId in required_class_index and confidence > confThreshold:
                w, h = int(det[2] * width), int(det[3] * height)
                x, y = int((det[0] * width) - w / 2), int((det[1] * height) - h / 2)
                boxes.append([x, y, w, h])
                classIds.append(classId)
                confidence_scores.append(float(confidence))

    # Apply Non-Max Suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidence_scores, confThreshold, nmsThreshold)

    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            color = [int(c) for c in colors[classIds[i]]]
            name = classNames[classIds[i]]
            detected_classNames.append(name)
            # Draw classname and confidence score
            cv2.putText(img, f'{name.upper()} {int(confidence_scores[i] * 100)}%',
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            # Draw bounding rectangle
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            detection.append([x, y, w, h, required_class_index.index(classIds[i])])

    return detected_classNames, img

def from_static_image(camera_id, timestamp):
    
    image = "image.jpg"
    
    input_size = 320
    img = cv2.imread(image)
    blob = cv2.dnn.blobFromImage(img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

    # Set the input of the network
    net.setInput(blob)
    layersNames = net.getLayerNames()
    outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]

    # Feed data to the network
    outputs = net.forward(outputNames)

    # Find the objects from the network output
    detected_classNames, output_img = postProcess(outputs, img)

    # Count the frequency of detected classes
    frequency = collections.Counter(detected_classNames)

    num_vehicles = 0
    for freq_item in frequency:
        num_vehicles += frequency[freq_item]

    # Display the output image
    #cv2.imshow("Detected Image", output_img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()

    # Print the detected class names
    #print("Detected Classes:")
    #for class_name in detected_classNames:
    #    print(class_name)
    
    global df
    
    distance = 
    
    new_row = {'Timestamp': timestamp, 'Camera_ID': camera_id, 'Vehicle_Count':num_vehicles}
    
    new_df = pd.DataFrame([new_row])

    # Concatenate the new DataFrame with the existing DataFrame
    df = pd.concat([df, new_df], ignore_index=True)
    

80 ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


**For one day**

In [ ]:
if __name__ == '__main__':

    
    now = datetime.now()

    # Calculate datetime for yesterday
    yesterday = now - timedelta(days=1)

    # Set the starting datetime for the interval
    current_datetime = datetime(yesterday.year, yesterday.month, yesterday.day, 0, 0)

    # Generate datetime values in 5-minute intervals until now
    while current_datetime <= now:
        #print(current_datetime)
        current_datetime += timedelta(minutes=5)
        #get_image(current_datetime)

        get_images(current_datetime)
    
    
        
    df.to_csv('./traffic_count.csv', index=False)

**For one hour**

In [11]:
if __name__ == '__main__':

    
    now = datetime.now()

    # Calculate datetime for 1 hour ago
    one_hour_ago = now - timedelta(hours=1)

    # Set the starting datetime for the interval
    current_datetime = datetime(one_hour_ago.year, one_hour_ago.month, one_hour_ago.day, one_hour_ago.hour, 0)

    # Generate datetime values in 1-hour intervals until now
    while current_datetime <= now:
        # Perform operations with current_datetime
        # get_image(current_datetime)
        get_images(current_datetime)
        print(current_datetime)
        current_datetime += timedelta(minutes=5)
    
    
        
    df.to_csv('./traffic_count.csv', index=False)  

2023-06-05 21:00:00
2023-06-05 21:05:00
2023-06-05 21:10:00
2023-06-05 21:15:00
2023-06-05 21:20:00
2023-06-05 21:25:00
2023-06-05 21:30:00
2023-06-05 21:35:00
2023-06-05 21:40:00
2023-06-05 21:45:00
2023-06-05 21:50:00
2023-06-05 21:55:00
2023-06-05 22:00:00
2023-06-05 22:05:00
2023-06-05 22:10:00
2023-06-05 22:15:00
2023-06-05 22:20:00
